In [3]:
# from sklearn.model_selection import train_test_split
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import numpy as np
import keras 
from numpy.lib.format import open_memmap

In [4]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# keras.backend.set_session(sess)

In [5]:
path_data = 'F:/Master Project/Dataset/Extract_data/25 joints/'
train_X = np.load(path_data+'train_x.npy', mmap_mode='r')
train_Y = np.load(path_data+'train_y.npy', mmap_mode='r')
print(train_X.shape)

(912031, 20, 75)


In [6]:
# train_X[100][10]

In [7]:
test_X = np.load(path_data+'test_x.npy', mmap_mode='r')
test_Y = np.load(path_data+'test_y.npy', mmap_mode='r')
print(test_X.shape)

(371045, 20, 75)


In [8]:
def create_model(num_frame, num_joint, num_output):
    model = Sequential()
    model.add(CuDNNLSTM(50, input_shape=(num_frame, num_joint),return_sequences=False))
    model.add(Dropout(0.4))#使用Dropout函数可以使模型有更多的机会学习到多种独立的表征
#     model.add(CuDNNLSTM(50, input_shape=(num_frame, num_joint)))
    model.add(Dense(64))
    model.add(Dropout(0.4))
    model.add(Dense(num_output, activation='softmax'))
    return model

In [9]:
load_model = False
max_frame = test_X.shape[1]
number_column = test_X.shape[2]
model = create_model(max_frame, number_column, 4)
start_epoch = 0

if load_model:
    weights_path = 'weights-improvement-04-0.75.hdf5'
    
    model.load_weights(weights_path)

sgd = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])    
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, )
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(train_X, train_Y, epochs=2,
          validation_data=(test_X,test_Y) , callbacks=callbacks_list, initial_epoch=start_epoch)

Train on 912031 samples, validate on 371045 samples
Epoch 1/2
912000/912031 [============================>.] - ETA: 0s - loss: 0.4680 - accuracy: 0.8131
Epoch 00001: saving model to weights-improvement-01-0.86.hdf5
912031/912031 [==============================] - 5059s 6ms/sample - loss: 0.4680 - accuracy: 0.8131 - val_loss: 0.3780 - val_accuracy: 0.8579
Epoch 2/2
 24448/912031 [..............................] - ETA: 1:00:10 - loss: 0.2371 - accuracy: 0.9204

In [ ]:
history.history